# 特征

1. 近期（3日内）有涨停（涨停或者>10%)
2. 上一周期30分钟RSI处于低位
3. 本周期30分钟确认低位成立，如长下影阳线、触及某根均线、阳线等。

<img src="https://images.jieyu.ai/images/202111/20211103225820.png" width="450"/>

In [4]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsi30, rsiday
from alpha.utils import math_round, buy_limit_price
from numpy.lib.stride_tricks import sliding_window_view

await init_notebook()

In [2]:
def buy_limit_trigger(code, name, bars, results, frame_type):
    """找到日线近期内出现涨停的股。时间由bars来控制"""
    close = bars["close"]
    
    blp = np.array([buy_limit_price(code, c0, bars[-1]['frame']) for c0 in close[:-1]])
    
    gaps = np.abs(blp - close[1:])
    counts = np.count_nonzero(gaps < 0.01)
    
    if counts > 0:
        results.append([name, code, counts])

def rsi_trigger(code, name, bars, results, frame_type):
    """30分钟RSI低位信号"""
    close = bars["close"]
    rsi = relative_strength_index(close, 6)
    # 前一周期低位
    prsi = rsi30.get_proba(code, rsi[-2])
    # 最后一周期收阳
    c0, o0, l0 = bars[-1]["close"], bars[-1]["open"], bars[-1]["low"]
    isbull = c0 > o0
    
    # 实体高度,只有当收阳时才有意义
    real_body = round(c0/o0 - 1, 3)
    
    # 下影线与实体之比，只有当收阳时才有意义
    if real_body != 0:
        shadow = round((o0/l0 - 1)/real_body, 3)
    else:
        shadow = 0

    if prsi <= 0.1 and isbull:
        results.append([name, code, bars[-1]["frame"], prsi, round(real_body,2), round(shadow,1)])
        
    return results

In [3]:
async def strategy():
    r1 = await scan(buy_limit_trigger, 10)
    codes = [item[1] for item in r1]
    r2 = await scan(rsi_trigger, 40, '30m', codes=codes)
    if len(r2):
        df = pd.DataFrame(r2, columns=["name", "code", "frame", "RSI", "real_body", "shadow"])
        display(df)
        await nb_say("涨停回调策略有新的信号.")

In [4]:
await scheduler(strategy)

=========== 13:15============
=========== 13:30============
=========== 13:45============
=========== 14:00============
=========== 14:15============
=========== 14:30============
=========== 14:45============


In [10]:
bars = await get_bars("000055.XSHE", 10, '30m', "2021-11-04 15:00")